In [1]:
import sqlite3
import pickle

from collections import defaultdict as dd

from tqdm import tqdm
from time import sleep

import pandas as pd
import numpy as np

from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [2]:
# source: https://realpython.com/python-web-scraping-practical-introduction/
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [3]:
# database connection
conn = sqlite3.connect('stocks_data/sp500.db')
c = conn.cursor()

# get tickers and names
symbols = pd.read_sql("SELECT DISTINCT(symbol) FROM fact_table", con=conn)
dates = []
names = [] 
for i, row in symbols.iterrows():
    s = row['symbol']
    d = pd.read_sql("SELECT min(date), max(date) FROM fact_table WHERE symbol='{}'".format(s), con=conn)
    n = pd.read_sql("SELECT name, sector FROM dimension_table WHERE symbol='{}'".format(s), con=conn)
    dates.append(np.array(d)[0])
    names.append(np.array(n)[0])
ticker_df = pd.concat((symbols, pd.DataFrame(names, columns=['name','sector']), pd.DataFrame(dates, columns=['startdate', 'enddate'])), axis=1)

In [4]:
# one off, import tesla
ticker, name, industry, sd, ed = np.array(ticker_df.iloc[3])
tesla_df = pd.read_csv('stocks_data/tesla.csv')
tesla_df['date']
name = 'tesla'
textd = dd(dict)
visited = set()

In [5]:
# non-tesla companies
ticker, name, industry, sd, ed = np.array(ticker_df.iloc[3])
textd = dd(dict)
visited = set()
# name = name.replace('.', '')
# name = name.replace(',', '')
name = 'amazon.com inc' # 'amazon.com inc', 'apple inc', 'disney', 'goldman sachs', 'tesla'

In [97]:
# api key
key = 'tTGUbrG64HtoMQraJqLRgh4K1CNDqAxD'

# iterate over articles in the month and if found a match, go to url and scrape data
for year in np.arange(2010, 2019):
    for month in np.arange(1, 13):
        print(year, month)
        link = 'https://api.nytimes.com/svc/archive/v1/' + str(year) + '/' + str(month) + '.json?api-key=' + key
        metadata = pd.read_json(link)
        
        # iterate over each article
        for article in metadata['response']['docs']:
            for line in article['keywords']:

                # match and never came across the article in the month of interest
                if name.lower() in line['value'].lower() and article['web_url'] not in visited:                
                    visited.add(article['web_url'])
                    date = article['pub_date'].split('T')[0]
                    
                    print(article['web_url'])
                    print(article['keywords'])
                    
                    # scrape text
                    try:
                        doc = BeautifulSoup(simple_get(article['web_url']), 'html.parser')
                        paragraphs = doc.findAll('p', itemprop='articleBody')
                        if not paragraphs:
                            print('no articlebody')
                            paragraphs = doc.findAll('p', attrs={'class': 'story-body-text story-content'})
                        if not paragraphs:
                            print('no story body text')
                            paragraphs = doc.findAll('p', attrs={'class': 'css-1ygdjhk evys1bk0'})
                        if not paragraphs:
                            print('no css tag')
                        for i in range(len(paragraphs)):
                            textd[article['pub_date'].split('T')[0]][i] = paragraphs[i].get_text().lower()
                        # debug
                        print(textd[article['pub_date'].split('T')[0]] != {})
                    except Exception as exc:
                        print('traceback.format_exc()')
                        print(exc)
                    # next article
                    break            

2010 1
https://gadgetwise.blogs.nytimes.com/2010/01/06/vudu-expands-its-service-to-more-tv-makers/
[{'rank': '11', 'name': 'organizations', 'value': 'Vudu Inc'}, {'rank': '4', 'name': 'organizations', 'value': 'LG Electronics'}, {'rank': '10', 'name': 'organizations', 'value': 'Vizio Inc'}, {'rank': '3', 'name': 'organizations', 'value': 'Boxee Inc'}, {'rank': '2', 'name': 'organizations', 'value': 'Amazon.com Inc'}, {'rank': '5', 'name': 'organizations', 'value': 'Netflix Inc'}, {'rank': '1', 'name': 'type_of_material', 'value': 'News'}, {'rank': '6', 'name': 'organizations', 'value': 'Roku'}, {'rank': '7', 'name': 'organizations', 'value': 'Roxio Inc'}, {'rank': '13', 'name': 'unknown', 'value': 'Netflix Inc|NFLX|NASDAQ'}, {'rank': '8', 'name': 'organizations', 'value': 'Sharp Corp'}, {'rank': '9', 'name': 'organizations', 'value': 'Toshiba Corporation'}, {'rank': '12', 'name': 'unknown', 'value': 'Amazon.com Inc|AMZN|NASDAQ'}]
True
https://gadgetwise.blogs.nytimes.com/2010/01/07/pla

[{'rank': '11', 'name': 'unknown', 'value': 'Amazon.com Inc|AMZN|NASDAQ'}, {'rank': '4', 'name': 'subject', 'value': 'Book Trade'}, {'rank': '10', 'name': 'organizations', 'value': 'Macmillan Publishers'}, {'rank': '3', 'name': 'subject', 'value': 'Amazon Kindle'}, {'rank': '2', 'name': 'persons', 'value': 'Mantel, Hilary'}, {'rank': '5', 'name': 'subject', 'value': 'Books and Literature'}, {'rank': '1', 'name': 'type_of_material', 'value': 'News'}, {'rank': '6', 'name': 'subject', 'value': 'Electronic Books and Readers'}, {'rank': '7', 'name': 'subject', 'value': 'Royalties'}, {'rank': '8', 'name': 'subject', 'value': 'Writing and Writers'}, {'rank': '9', 'name': 'organizations', 'value': 'Amazon.com Inc'}]
True
https://www.nytimes.com/2010/02/04/technology/04amazon.html
[{'rank': '1', 'is_major': 'N', 'name': 'subject', 'value': 'Computers and the Internet'}, {'rank': '2', 'is_major': 'N', 'name': 'subject', 'value': 'Amazon Kindle'}, {'rank': '3', 'is_major': 'N', 'name': 'subject',

[{'rank': '11', 'name': 'unknown', 'value': 'Apple Inc|AAPL|NASDAQ'}, {'rank': '4', 'name': 'subject', 'value': 'Books and Literature'}, {'rank': '10', 'name': 'unknown', 'value': 'Amazon.com Inc|AMZN|NASDAQ'}, {'rank': '3', 'name': 'subject', 'value': 'Book Trade'}, {'rank': '2', 'name': 'subject', 'value': 'Blogs and Blogging (Internet)'}, {'rank': '5', 'name': 'subject', 'value': 'Computers and the Internet'}, {'rank': '1', 'name': 'type_of_material', 'value': 'News'}, {'rank': '6', 'name': 'subject', 'value': 'iPad'}, {'rank': '7', 'name': 'subject', 'value': 'iPhone'}, {'rank': '8', 'name': 'organizations', 'value': 'Amazon.com Inc'}, {'rank': '9', 'name': 'organizations', 'value': 'Apple Inc'}]
True
https://bits.blogs.nytimes.com/2010/03/05/apple-ipad-arrives-in-stores-on-april-3/
[{'rank': '11', 'name': 'subject', 'value': 'Wireless Communications'}, {'rank': '4', 'name': 'subject', 'value': 'Amazon Kindle'}, {'rank': '10', 'name': 'subject', 'value': 'Shopping and Retail'}, {'r

[{'value': 'Cloud Computing', 'is_major': 'N', 'rank': '1', 'name': 'subject'}, {'value': 'Computers and the Internet', 'is_major': 'N', 'rank': '2', 'name': 'subject'}, {'value': 'Enterprise Computing', 'is_major': 'N', 'rank': '3', 'name': 'subject'}, {'value': 'Data Centers', 'is_major': 'N', 'rank': '4', 'name': 'subject'}, {'value': 'Amazon.com Inc', 'is_major': 'N', 'rank': '5', 'name': 'organizations'}, {'value': 'Microsoft Corp', 'is_major': 'N', 'rank': '6', 'name': 'organizations'}, {'value': 'Google Inc', 'is_major': 'N', 'rank': '7', 'name': 'organizations'}, {'value': 'AT&T Corp', 'is_major': 'N', 'rank': '8', 'name': 'organizations'}]
no articlebody
True
https://dealbook.nytimes.com/2010/04/20/another-view-peering-clearly-at-the-future/
[{'rank': '11', 'name': 'subject', 'value': 'Start-ups'}, {'rank': '4', 'name': 'subject', 'value': 'Data Storage'}, {'rank': '21', 'name': 'unknown', 'value': 'Amazon.com Inc|AMZN|NASDAQ'}, {'rank': '10', 'name': 'subject', 'value': 'Soft

[{'value': 'Antitrust Actions and Laws', 'is_major': 'N', 'rank': '1', 'name': 'subject'}, {'value': 'iTunes', 'is_major': 'N', 'rank': '2', 'name': 'organizations'}, {'value': 'Apple Inc', 'is_major': 'N', 'rank': '3', 'name': 'organizations'}, {'value': 'Amazon.com Inc', 'is_major': 'N', 'rank': '4', 'name': 'organizations'}, {'value': 'Justice Department', 'is_major': 'N', 'rank': '5', 'name': 'organizations'}, {'value': 'Computers and the Internet', 'is_major': 'N', 'rank': '6', 'name': 'subject'}, {'value': 'Music', 'is_major': 'N', 'rank': '7', 'name': 'subject'}, {'value': 'Recordings and Downloads (Audio)', 'is_major': 'N', 'rank': '8', 'name': 'subject'}]
no articlebody
True
https://bits.blogs.nytimes.com/2010/05/27/olpc-partners-with-marvell-to-launch-100-tablet/
[{'rank': '11', 'name': 'organizations', 'value': 'Apple Inc'}, {'rank': '4', 'name': 'subject', 'value': 'Computers and the Internet'}, {'rank': '10', 'name': 'organizations', 'value': 'Amazon.com Inc'}, {'rank': '1

...

[{'name': 'subject', 'value': 'United States International Relations', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Computers and the Internet', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Cyberwarfare and Defense', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Espionage and Intelligence Services', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 5, 'major': 'N'}, {'name': 'organizations', 'value': 'Apple Inc', 'rank': 6, 'major': 'N'}]
no articlebody
no story body text
True
https://www.nytimes.com/2018/10/09/technology/amazon-workers-pay-raise.html
[{'name': 'subject', 'value': 'Wages and Salaries', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Labor and Jobs', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Bonuses', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Stock Options and Purchase Plans', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 5, '

[{'name': 'subject', 'value': 'Book Trade and Publishing', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'E-Commerce', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'AbeBooks Inc', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Demonstrations, Protests and Riots', 'rank': 5, 'major': 'N'}]
no articlebody
no story body text
True
https://www.nytimes.com/2018/11/06/nyregion/amazon-long-island-city.html
[{'name': 'subject', 'value': 'Subways', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 2, 'major': 'N'}, {'name': 'glocations', 'value': 'Long Island City (Queens, NY)', 'rank': 3, 'major': 'N'}]
no articlebody
no story body text
True
https://www.nytimes.com/2018/11/06/opinion/amazon-hq2-split.html
[{'name': 'subject', 'value': 'Labor and Jobs', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Relocation of Business', 'rank': 2, 'maj

https://www.nytimes.com/2018/11/14/style/jonathan-adler-amazon.html
[{'name': 'subject', 'value': 'E-Commerce', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Interior Design and Furnishings', 'rank': 2, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 3, 'major': 'N'}, {'name': 'persons', 'value': 'Adler, Jonathan (1966- )', 'rank': 4, 'major': 'N'}]
no articlebody
no story body text
True
https://www.nytimes.com/2018/11/15/opinion/military-veterans-colleges-universities.html
[{'name': 'subject', 'value': 'Colleges and Universities', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Veterans', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Graduation Rates', 'rank': 3, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Capitalism (Theory and Philosophy)', 'rank': 5, 'major': 'N'}, {'name': 'subject', 'value': 'Corporations', 'rank': 6, 'major': 'N'}, {'name': 'sub

[{'name': 'subject', 'value': 'Athletics and Sports', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Media', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Mergers, Acquisitions and Divestitures', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Cable Television', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 5, 'major': 'N'}, {'name': 'organizations', 'value': '21st Century Fox', 'rank': 6, 'major': 'N'}]
no articlebody
no story body text
True
https://www.nytimes.com/2018/11/21/world/australia/amazon-australia-access-website.html
[{'name': 'subject', 'value': 'E-Commerce', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Shopping and Retail', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Black Friday and Cyber Monday (Shopping)', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'International Trade and World Market', 'rank': 4, 'major': 'N'}, {'name': 'subject', 'value': 'Delivery Services', 'rank

[{'name': 'organizations', 'value': 'Google Inc', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Amazon.com Inc', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Computers and the Internet', 'rank': 3, 'major': 'N'}, {'name': 'subject', 'value': 'Real Estate (Commercial)', 'rank': 4, 'major': 'N'}, {'name': 'organizations', 'value': 'Taconic Investment Partners', 'rank': 5, 'major': 'N'}, {'name': 'persons', 'value': 'Hoylman, Brad M', 'rank': 6, 'major': 'N'}, {'name': 'persons', 'value': 'Johnson, Corey', 'rank': 7, 'major': 'N'}, {'name': 'glocations', 'value': 'Chelsea (Manhattan, NY)', 'rank': 8, 'major': 'N'}, {'name': 'glocations', 'value': 'New York City', 'rank': 9, 'major': 'N'}]
no articlebody
no story body text
True
https://www.nytimes.com/2018/12/16/business/economy/nashville-birmingham-amazon.html
[{'name': 'subject', 'value': 'Labor and Jobs', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'Relocation of Business', 'rank': 2, 'major': 'N'},

In [98]:
# write to file with pickle (as binary)
name='amazon' # amazon, apple, disney, goldman_sachs, tesla
f = open('news_data/' + str(name) + '.pickle', 'wb')
newData = pickle.dump(textd, f)
# f.write(newData)
f.close()

In [103]:
# open file with pickle (as binary)
name='amazon'
f = open('news_data/' + str(name) + '.pickle', 'rb')
raw_articles = pickle.load(f)
f.close()